## Test and overview of the `mdf_reader` tool

First clone the [gitlab repository](https://git.noc.ac.uk/brecinosrivas/mdf_reader) and modify the path in `sys.path.append()` with the directory path where you store the `mdf_reader` repository.

In [ ]:
from __future__ import annotations

import os
import sys

sys.path.append("/home/bea/")

The `mdf_reader` is a python3 tool designed to read data files compliant with a user specified [data
model](https://cds.climate.copernicus.eu/toolbox/doc/how-to/15_how_to_understand_the_common_data_model/15_how_to_understand_the_common_data_model.html).

It was developed with the initial idea to read the [IMMA](https://icoads.noaa.gov/e-doc/imma/R3.0-imma1.pdf) data format, but it was further enhanced to account for other meteorological data formats.

Lets see an example for a typical file from [ICOADSv3.0.](https://icoads.noaa.gov/r3.html). We pick an specific montly output for a Source/Deck. In this case data from the Marine Meterological Journals data set SID/DCK: **125-704 for Oct 1878.**

The `.imma` file looks like this:

In [ ]:
from __future__ import annotations

import mdf_reader
import pandas as pd

data_path = os.path.join(
    "~/c3s_work", "mdf_reader/tests/data/125-704_1878-10_subset.imma"
)
data_ori = pd.read_table(data_path)

In [ ]:
data_ori.head()

Very messy to just read into python!

This is why we need the `mdf_reader` tool, to helps us put those imma files in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) format. For that we need need a **schema**.

A **schema** file gathers a collection of descriptors that enable the `mdf_reader` tool to access the content
of a `data model/ schema` and extract the sections of the raw data file that contains meaningful information. These **schema files** are the `bones` of the data model, basically `.json` files outlining the structure of the incoming raw data.

The `mdf_reader` takes this information and translate the characteristics of the data to a python pandas dataframe.

The tool has several **schema** templates build in.

In [ ]:
template_names = mdf_reader.schemas.templates()

In [ ]:
template_names

As well as templates for `code_tables` which are `.json` files containing keys to describe complex metereological variables like weather forcast or sea state conditions.

In [ ]:
template_tables = mdf_reader.code_tables.templates()
template_tables

**Schemas** can be desinged to be deck specific like the example below

In [ ]:
schema = "imma1_d704"

data_file_path = "/home/bea/c3s_work/mdf_reader/tests/data/125-704_1878-10_subset.imma"

data = mdf_reader.read(data_file_path, data_model=schema)

A new **schema** can be build for a particular deck and source as shown in this notebook. The `imma1_d704` schema was build upon the `imma1` schema/data model but extra sections have been added to the `.json` files to include suplemental data from ICOADS documentation. This is a snapshot of the data inside the `imma1_d704.json`.

```
"c99_journal": {
            "header": {"sentinal": "1", "field_layout":"fixed_width","length": 117},
            "elements": {
              "sentinal":{
                  "description": "Journal header record identifier",
                  "field_length": 1,
                  "column_type": "str"
              },
              "reel_no":{
                  "description": "Microfilm reel number. See if we want the zero padding or not...",
                  "field_length": 3,
                  "column_type": "str",
                  "LMR6": true
              }

```

Now metadata information can be extracted as a component of the padas dataframe.

In [ ]:
data.data.c99_journal

To learn how to construc a schema or data model for a particular deck/source, visit this other tutorial: [create_data_model.ipynb](https://git.noc.ac.uk/brecinosrivas/mdf_reader/-/blob/master/docs/notebooks/create_data_model.ipynb)